In [169]:
# Import necessary modules from PyTorch
import torchvision
import torch
from torchvision import transforms, datasets

# Import necessary modules for Neural Network
import torch.nn as nn
import torch.optim as optim

## CNN Model Architecture

In [170]:
# Define custom Convolution Neural Network
# Simple CNN architecture with two convolutional layers followed by max pooling, two fully connected layers, and a dropout layer for regularization.
class CNN(nn.Module):
    def __init__(self, num_channels=3, num_out_ch=[8, 16], img_w=100, img_h=100, dropout=0.5, num_classes=102):
        super(CNN, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=num_channels, out_channels=num_out_ch[0], kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=num_out_ch[0], out_channels=num_out_ch[1], kernel_size=3, stride=1, padding=1)

        # Max pooling layer: down-sample an image by applying max filer to subregion
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        # Fully connected layers
        self.fc = nn.Linear(in_features = 16 * 56 * 56, out_features=num_classes)
        # self.fc1 = nn.Linear(32 * 56 * 56, 512)  # Adjust input size based on image dimensions
        # self.fc2 = nn.Linear(512, num_classes)

        # Dropout layer to prevent overfitting
        self.dropout = nn.Dropout(p=dropout)

    
    # Defines the forward pass of the network, where input data x is passed through each layer sequentially.
    def forward(self, x):

        # Convolutional layers with ReLU activation and max pooling
        x = self.pool(nn.ReLU()(self.conv1(x)))
        
        x = self.pool(nn.ReLU()(self.conv2(x)))
        x = self.dropout(x)

        # Flatten the output from convolutional layers
        # x = x.view(-1, 32 * 56 * 56)  # Adjust size based on image dimensions
        x = x.view(x.size(0), -1)  # Adjust size based on image dimensions

        # Fully connected layers with dropout
        x = self.fc(x.reshape(x.shape[0], -1))
        # x = self.dropout(x)
        # x = self.dropout(nn.ReLU()(self.fc1(x)))
        # x = self.fc2(x)
        return x

### Set Training Parameters, Device, Model, Optimizer, and Loss Function

In [202]:
# Hyperparameters
NUM_OUT_CH = [8, 16]
IMAGE_W = 200
IMAGE_H = 200
BATCH_SIZE = 64
NUM_WORKERS = 8
NUM_EPOCHS = 100  # Number of training epochs
LR = 0.001
WEIGHT_DECAY = 0.0001
DROPOUT = 0.1

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create an instance of the CNN model
model = CNN(num_channels=3, num_out_ch=NUM_OUT_CH, img_w=IMAGE_W, img_h=IMAGE_H, dropout=DROPOUT, num_classes=102).to(device)  # 102 classes for Flowers102 dataset

# Define the loss function
loss_fn = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

## Preprosessing Data

In [183]:
# Data augmentation and normalization
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),  # Randomly crop and resize the image
    transforms.RandomHorizontalFlip(),   # Randomly flip the image horizontally
    transforms.RandomRotation(10),       # Randomly rotate the image by up to 10 degrees
    transforms.ColorJitter(0.2, 0.2, 0.2),  # Randomly adjust brightness, contrast, and saturation
    transforms.ToTensor(),               # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
])

val_transform = transforms.Compose([
    transforms.Resize(256),              # Resize the image to 256x256
    transforms.CenterCrop(224),          # Crop the center of the image to 224x224
    transforms.ToTensor(),               # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
])

# Define dataset root directory
data_dir = 'dataset_flower102/'

In [184]:
# Apply transformations to the dataset during data loading
train_dataset = datasets.Flowers102(root=data_dir, split='train', transform=train_transform, download=True)
valid_dataset = datasets.Flowers102(root=data_dir, split='val', transform=val_transform, download=True)
# test_dataset = datasets.Flowers102(root=data_dir, split='test', transform=data_transforms, download=True)

In [185]:
# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, shuffle=False)

## CNN Model Training

### Model Training and Validation

In [203]:
# Load the best trained model along with training and validation loss values
# checkpoint = torch.load('best_trained_model.pth')
# best_model_state = checkpoint['model_state_dict']
# train_loss = checkpoint['train_loss']
# val_loss = checkpoint['val_loss']
# model.load_state_dict(best_model_state) # Set the model state and loss values before resuming training (Need to comment out the train_loss and val_loss)

best_val_loss = float('inf')  # Initialize the best validation loss with a large value (float('inf'))
patience = 30  # Number of epochs to wait before stopping if validation loss doesn't improve

for epoch in range(NUM_EPOCHS):
    
    # Train the model
    model.train() # Set the model to training mode
    train_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device) # Move images and labels to GPU
        optimizer.zero_grad() # Zero the parameter gradients
        outputs = model.forward(images) # Forward pass
        loss = loss_fn(outputs, labels) # Calculate the loss
        loss.backward() # Backward pass
        optimizer.step() # Optimize

        train_loss += loss.item() * images.size(0) #  scalar value of the loss tensor for the current batch * the batch size to account for the loss per sample in the batch
        _, predicted = outputs.max(1) # Returns a tuple containing the maximum value along the specified dimension (class probabilities for each sample in the batch) and index of the max value
        total += labels.size(0) # Accumulates the total number of sample seen during training
        correct += predicted.eq(labels).sum().item() # Accumulates the total number of correct predictions over all batches.
    
    # Calculate training loss and accuracy
    train_loss = train_loss / len(train_loader.dataset)
    train_accuracy = 100.0 * correct / total

    # Print training loss and accuracy
    print(f'Epoch {epoch + 1}/{NUM_EPOCHS}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%')



    # Validate the model
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device) # Move images and labels to GPU
            outputs = model.forward(images)  # Forward pass
            loss = loss_fn(outputs, labels)  # Calculate the loss

            val_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    # Calculate validation loss and accuracy
    val_loss = val_loss / len(valid_loader.dataset)
    val_accuracy = 100.0 * correct / total

    # Print validation loss and accuracy
    print(f'Epoch {epoch + 1}/{NUM_EPOCHS}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')

    # Check if validation loss has improved
    if val_loss < best_val_loss:
        print("Creating new checkpoint for best model...")
        best_val_loss = val_loss
        patience = 30  # Reset patience if validation loss improves
        best_model_state = model.state_dict()  # Save the best model state
        current_val__loss = val_loss
        current_train_loss = train_loss
    else:
        patience -= 1
        if patience == 0:
            print("Early stopping...")
            break

# Save the best trained model along with training and validation loss values
torch.save({
    'model_state_dict': best_model_state,
    'train_loss': current_train_loss,
    'val_loss': current_val__loss
}, 'best_trained_model.pth')

# save the completed model training
finish_model_state = model.state_dict()
torch.save({
    'model_state_dict': finish_model_state,
    'train_loss': train_loss,
    'val_loss': val_loss
}, 'finish_trained_model.pth')


Epoch 1/100, Training Loss: 6.2501, Training Accuracy: 1.47%
Epoch 1/100, Validation Loss: 4.5504, Validation Accuracy: 4.02%
Creating new checkpoint for best model...
Epoch 2/100, Training Loss: 4.4031, Training Accuracy: 4.12%
Epoch 2/100, Validation Loss: 4.1405, Validation Accuracy: 6.67%
Creating new checkpoint for best model...
Epoch 3/100, Training Loss: 4.1131, Training Accuracy: 6.76%
Epoch 3/100, Validation Loss: 3.9450, Validation Accuracy: 8.82%
Creating new checkpoint for best model...
Epoch 4/100, Training Loss: 3.9317, Training Accuracy: 7.94%
Epoch 4/100, Validation Loss: 3.7902, Validation Accuracy: 11.08%
Creating new checkpoint for best model...
Epoch 5/100, Training Loss: 3.7708, Training Accuracy: 11.47%
Epoch 5/100, Validation Loss: 3.6297, Validation Accuracy: 13.43%
Creating new checkpoint for best model...
Epoch 6/100, Training Loss: 3.6379, Training Accuracy: 15.00%
Epoch 6/100, Validation Loss: 3.4777, Validation Accuracy: 16.96%
Creating new checkpoint for b